# CP2 - Relatório com Análise dos Dados Automatizada

## Integrantes
- **Nikolas Rodrigues Moura dos Santos** - RM: 551566
- **Rodrigo Brasileiro** - RM: 98952
- **Thiago Jardim de Oliveira** RM: 551624

Aqui está a reformulação do texto em cada um dos tópicos:

### Objetivo do Trabalho
Este projeto tem como meta desenvolver um sistema automatizado de análise de dados utilizando a linguagem Python. Serão usadas bibliotecas como pandas, matplotlib e seaborn para análise e visualização de dados, além de jinja2 para criar relatórios HTML com estilo Bootstrap. O sistema processa um dataset e automaticamente gera um relatório com estatísticas e gráficos principais, facilitando o processo de análise exploratória.

### Estrutura do Código
#### Bibliotecas Utilizadas
- **pandas**: Usada para tratar e analisar os dados organizados em dataframes.
- **matplotlib e seaborn**: Responsáveis pela criação de gráficos para visualização dos dados.
- **jinja2**: Utilizada como motor de templates para gerar os relatórios HTML.
- **os, base64 e BytesIO**: Empregados para manipulação de arquivos e conversão de gráficos em Base64, facilitando sua inclusão no HTML gerado.

### Passo a Passo do Código
#### Configuração do Template HTML:
O código monta um template HTML usando o Bootstrap para estilização. Esse template apresenta as estatísticas e gráficos em cards organizados, proporcionando uma apresentação clara e ordenada das informações.

#### Função `summarize_df(df)`:
Esta função calcula as principais estatísticas sobre o dataset, como:
- Total de variáveis.
- Número de registros.
- Contagem de células nulas.
- Número de linhas duplicadas.
- Quantidade de variáveis categóricas e numéricas.
Ela retorna essas estatísticas em um dicionário chamado `summary`.

#### Função `plot_64(fig)`:
Responsável por converter os gráficos gerados em strings Base64 para que possam ser incorporados diretamente no HTML. Isso garante que os gráficos sejam exibidos corretamente, independentemente do ambiente onde o relatório é visualizado.

#### Função `create_plot(df)`:
Gera gráficos específicos para os dados numéricos (histogramas) e categóricos (gráficos de contagem), utilizando seaborn para a formatação visual e matplotlib para a renderização.

#### Função `to_html(df)`:
Esta função utiliza o jinja2 para renderizar o template HTML, incorporando as estatísticas e gráficos obtidos. O resultado é um relatório estilizado e salvo no ambiente, pronto para visualização.

#### Execução Principal (`if __name__ == '__main__'`):
Carrega um dataset (como o *pokemonDB_dataset.csv*) e gera o relatório em HTML através das funções anteriores. O relatório é exibido diretamente no notebook Colab por meio de um IFrame.

### Estrutura do Relatório
O relatório final em HTML é composto por várias seções, incluindo:
- **Título e Sumário**: Apresenta informações como número de linhas, colunas, variáveis, células faltantes e registros duplicados.
- **Tipos de Dados**: Exibe os tipos de dados de cada coluna.
- **Tipos de Variáveis**: Informa quantas variáveis são numéricas e quantas são categóricas.
- **Valores Faltantes**: Mostra o total de valores ausentes por coluna.
- **Gráficos**: Inclui gráficos para análise exploratória das variáveis, organizados em um layout de cards responsivos.

### Como Executar
Antes de rodar o código no Colab, verifique se as bibliotecas necessárias estão instaladas. Execute o código e carregue o dataset desejado. Se for usar um dataset diferente, modifique a linha que carrega o arquivo: `df = pd.read_csv('/content/nome-do-dataset.csv')`. Após a execução, o relatório será gerado e exibido diretamente no notebook.

### Conclusão
O projeto demonstra como a análise exploratória de dados pode ser automatizada de maneira prática e atrativa. A combinação do pandas para manipulação de dados, das bibliotecas de visualização e do uso de jinja2 para geração de relatórios em HTML resulta em relatórios ricos e acessíveis visualmente, facilitando a compreensão das informações.

In [1]:
# Instalação de bibliotecas necessárias
!pip install jinja2
!pip install matplotlib seaborn pandas


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from jinja2 import Environment, FileSystemLoader
import os
import base64
from io import BytesIO

# Configuração do ambiente para Jinja2 no Colab
!mkdir -p /content/templates
template_html = """
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Relatório com Análise dos Dados Automatizada</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
    <div class="container my-5">
        <h1 class="text-center mb-4">Relatório com Análise dos Dados Automatizada</h1>
        <div class="card mb-4">
            <div class="card-body">
                <h2 class="card-title">Sumário</h2>
                <p><strong>Número de Linhas:</strong> {{ summary.shape[0] }}</p>
                <p><strong>Número de Colunas:</strong> {{ summary.shape[1] }}</p>
                <p><strong>Número de Variáveis:</strong> {{ summary.num_var }}</p>
                <p><strong>Número de Observações:</strong> {{ summary.num_obs }}</p>
                <p><strong>Células Faltantes:</strong> {{ summary.cels_faltantes }}</p>
                <p><strong>Linhas Duplicadas:</strong> {{ summary.linhas_duplicadas }}</p>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Tipos de Data</h3>
                <ul class="list-group">
                    {% for column, dtype in summary.dtypes.items() %}
                    <li class="list-group-item">{{ column }}: {{ dtype }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Tipos de Variáveis</h3>
                <p><strong>Numérico:</strong> {{ summary.numeric_vars }}</p>
                <p><strong>Categórico:</strong> {{ summary.categorical_vars }}</p>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Valores Faltantes (null)</h3>
                <ul class="list-group">
                    {% for column, missing in summary.valores_faltantes.items() %}
                    <li class="list-group-item">{{ column }}: {{ missing }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>

        <h2 class="mt-5">Gráficos</h2>
        <div class="row">
            {% for plot in plots %}
            <div class="col-md-6 mb-4">
                <div class="card">
                    <img src="data:image/png;base64,{{ plot }}" class="card-img-top" alt="Plot">
                </div>
            </div>
            {% endfor %}
        </div>
    </div>

    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</body>
</html>
"""

# Salvando o template HTML no sistema de arquivos do Colab
with open('/content/templates/report_template.html', 'w') as file:
    file.write(template_html)

# Função para gerar resumo do dataframe
def summarize_df(df):
    num_var = len(df.columns)
    num_obs = len(df)
    cels_faltantes = df.isnull().sum().sum()
    linhas_duplicadas = df.duplicated().sum()

    numeric_vars = len(df.select_dtypes(include=np.number).columns)
    categorical_vars = len(df.select_dtypes(include='object').columns)

    summary = {
        'shape': df.shape,
        'num_var': num_var,
        'num_obs': num_obs,
        'cels_faltantes': cels_faltantes,
        'linhas_duplicadas': linhas_duplicadas,
        'numeric_vars': numeric_vars,
        'categorical_vars': categorical_vars,
        'dtypes': df.dtypes.to_dict(),
        'valores_faltantes': df.isnull().sum().to_dict(),
        'duplicados': df.duplicated().sum(),
        'status_descritiveis': df.describe().to_dict()
    }
    return summary

# Função para converter gráficos para Base64
def plot_64(fig):
    buf = BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return img_base64

# Função para gerar gráficos e retornar como Base64
def create_plot(df):
    plots = []

    # Histogramas para variáveis numéricas
    for column in df.select_dtypes(include=np.number).columns:
        fig, ax = plt.subplots()
        sns.histplot(df[column], kde=True, ax=ax)
        plt.title(f'Histograma de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Gráficos de contagem para variáveis categóricas
    for column in df.select_dtypes(include='object').columns:
        fig, ax = plt.subplots()
        sns.countplot(y=df[column], ax=ax)
        plt.title(f'Countplot de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    return plots

# Função para gerar relatório HTML
def to_html(df):
    env = Environment(loader=FileSystemLoader('/content/templates'))
    template = env.get_template('report_template.html')

    summary = summarize_df(df)
    plots = create_plot(df)

    html_content = template.render(summary=summary, plots=plots)

    # Salvar o relatório HTML no Colab
    report_path = '/content/relatorio.html'
    with open(report_path, 'w') as f:
        f.write(html_content)

    print(f'Report saved at: {report_path}')
    return report_path

if __name__ == '__main__':
    df = pd.read_csv('/content/pokemonDB_dataset.csv')
    report_path = to_html(df)

    # Exibir o relatório HTML no Colab
    from IPython.display import IFrame
    IFrame(report_path, width=800, height=600)


Report saved at: /content/relatorio.html
